In [ ]:
import numpy as np
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
from pyts.image import RecurrencePlot
from PIL import Image
import matplotlib.pyplot as plt
import os

data_paths = [['DataSets/mitbih_train_smote.csv','Datasets/mitbih_fusion_smote_227/train/'],['DataSets/mitbih_test.csv','Datasets/mitbih_fusion_smote_227/test/',]]

for path in data_paths:
    all_temp_serie = np.genfromtxt(path[0], delimiter=',')

    for i,temp_serie in enumerate(all_temp_serie):
        y = temp_serie[-1]
        x = np.delete(temp_serie,-1)
        x = np.concatenate((x,np.zeros(40)))
        x = np.expand_dims(x,axis=0)

        gaf = GramianAngularField(method='summation')
        mtf = MarkovTransitionField(n_bins=10,strategy='uniform')
        rp = RecurrencePlot(threshold=0.1,percentage=10)

        gaf_img_serie = (gaf.fit_transform(x) + 1) / 2
        mtf_img_serie = mtf.fit_transform(x)
        rp_img_serie = rp.fit_transform(x)

        img_serie = np.stack((gaf_img_serie,mtf_img_serie,rp_img_serie),axis=1)
        img_serie = np.moveaxis(img_serie,1,-1)

        save_path = path[1] + str(int(y)) + '/'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        img = Image.fromarray((img_serie.squeeze()*255).astype(np.uint8),'RGB')
        name = str(i) +'_gmr_fusion_' + 'class_' + str(int(y)) + '.jpg'
        img.save(save_path + name, 'JPEG')